<a href="https://colab.research.google.com/github/MahdiZaman/Spring2019-CAP5610/blob/master/hw1p3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from keras.datasets import mnist
(train_images_original, train_labels_original), (test_images_original, test_labels_original) = mnist.load_data()


Using TensorFlow backend.


In [0]:
# Preparing the Train and Test Images (Adding a column of 1s for biases)
nfeatures = 28*28
nclasses = 10
ntrainsamples = train_images_original.shape[0] #60000
ntestsamples = test_images_original.shape[0]   #10000

#normalizing inputs
train_images = train_images_original.reshape((ntrainsamples, nfeatures))
train_images = train_images.astype('float32') / 255 
test_images = test_images_original.reshape((ntestsamples, nfeatures))
test_images = test_images.astype('float32') / 255

#adding a column of 1s for bias calculation
train_images_b = np.c_[np.ones((ntrainsamples, 1)), train_images]
test_images_b = np.c_[np.ones((ntestsamples,1)), test_images]

# One hot encoding for train and test labels
train_labels = np.eye(nclasses)[train_labels_original]
test_labels = np.eye(nclasses)[test_labels_original]

In [0]:
def softmax(z,n):
#     z -= np.max(z)
    mx = np.max(z, axis=1).reshape((n,1))
    a = (np.exp(z-mx).T/ np.sum(np.exp(z-mx), axis=1)).T
    return a
#   return np.exp(z) / np.sum(np.exp(z), axis=0)

In [0]:
def cce_loss(y, y_hat):
    return -np.mean(y * np.log(y_hat))

In [0]:
def forwardprop(x, weight, ninput):
    xw = x.dot(weight)
    A = softmax(xw,ninput)
    return A

In [0]:
def deriv(train_images, train_labels, a , minibatch):
    dz = (a - train_labels) / minibatch
    dw = np.dot(train_images_m.T , dz)
    return dw

In [7]:
## Training the model

#initializing Weights and Bias(Bias added as a column of 1 on the left in weight array)

initial_weight = np.zeros((nfeatures+1, nclasses))
# print(initial_weight.shape)

weight = initial_weight
weight_path_mgd = []
weight_path_mgd.append(weight)

epochs = 20 #number of epochs
minibatchsize = 40 #number of samples in minibatch
lr = 0.01 #Learning Rate

for epoch in range(epochs):
    #shuffling the samples
    m = np.random.permutation(ntrainsamples)
    train_images_b = train_images_b[m]
    train_labels = train_labels[m]
    
    #minibatch SGD
    for i in range(0, ntrainsamples, minibatchsize):
        train_images_m = train_images_b[i : i+minibatchsize] 
        train_labels_m = train_labels[i : i+minibatchsize]
    
        A = forwardprop(train_images_m, weight, minibatchsize)
        loss = cce_loss(train_labels_m, A)
        
        # Updating Weights and Biases
        dw = deriv(train_images_m, train_labels_m, A , minibatchsize)
        weight = weight - lr*dw
        weight_path_mgd.append(weight)
# print(len(weight_path_mgd))
    print(('Epoch {}/{} --- Training Loss {}'.format(epoch+1 , epochs, np.mean(loss))))

Epoch 1/20 --- Training Loss 0.060122044491943576
Epoch 2/20 --- Training Loss 0.05020662316250966
Epoch 3/20 --- Training Loss 0.03483933942892684
Epoch 4/20 --- Training Loss 0.04602639749953889
Epoch 5/20 --- Training Loss 0.03787626744704162
Epoch 6/20 --- Training Loss 0.02024916707055842
Epoch 7/20 --- Training Loss 0.06307763536831894
Epoch 8/20 --- Training Loss 0.03025975230207153
Epoch 9/20 --- Training Loss 0.02740699543535266
Epoch 10/20 --- Training Loss 0.029575026114214083
Epoch 11/20 --- Training Loss 0.02548737363698044
Epoch 12/20 --- Training Loss 0.040355299880436436
Epoch 13/20 --- Training Loss 0.06335321517937123
Epoch 14/20 --- Training Loss 0.018284982792312155
Epoch 15/20 --- Training Loss 0.02661858610654939
Epoch 16/20 --- Training Loss 0.02303173832916223
Epoch 17/20 --- Training Loss 0.03532032434141179
Epoch 18/20 --- Training Loss 0.038613572141493975
Epoch 19/20 --- Training Loss 0.021854112418767703
Epoch 20/20 --- Training Loss 0.024526112184580243


In [8]:
## Testing the model

test_label_predicted = np.argmax(forwardprop(test_images_b, weight, test_labels_original.shape[0]) , axis = 1)
accuracy = np.sum(test_label_predicted == test_labels_original)/test_labels_original.shape[0] 

# t = 200
# print(test_labels_original[t])
# print(test_label_predicted[t])
print("Accuracy is {}".format(accuracy))


Accuracy is 0.9193
